# Reference materials
0. [Vector Representations of Words](https://www.tensorflow.org/tutorials/word2vec)
1. [Word2Vec Tutorial Part 1 - The Skip-Gram Model](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)
2. [Word2Vec Tutorial Part 2 - Negative Sampling](http://mccormickml.com/2017/01/11/word2vec-tutorial-part-2-negative-sampling/)
3. [From word2vec to doc2vec: an approach driven by Chinese restaurant process](https://medium.com/kifi-engineering/from-word2vec-to-doc2vec-an-approach-driven-by-chinese-restaurant-process-93d3602eaa31)
4. [Doc2Vec tutorial using Gensim](https://medium.com/@klintcho/doc2vec-tutorial-using-gensim-ab3ac03d3a1)
5. [Sentiment Analysis Using Doc2Vec](http://linanqiu.github.io/2015/10/07/word2vec-sentiment/)
6. [Understanding Convolutional Neural Networks for NLP](http://www.wildml.com/2015/11/understanding-convolutional-neural-networks-for-nlp/)
7. [Implementing a CNN for Text Classification in TensorFlow](http://www.wildml.com/2015/12/implementing-a-cnn-for-text-classification-in-tensorflow/)
8. [cnn-text-classification-github](https://github.com/cahya-wirawan/cnn-text-classification-tf)
9. [word2vec - Google Code Archive](https://code.google.com/archive/p/word2vec/) - need to download pretrained word2vec
10. [Using pretrained word embeddings](https://github.com/dennybritz/cnn-text-classification-tf/issues/17)

# How to do document clustering with word2vec and doc2vec?

**Steps:**
1. Train a word2vec model to get word vectors
    - Made word2vec_starter.py to work
    - Question: How to feed the embeddings to the doc2vec model?
        - Read in pre-trained embeddings
2. Get doc2vec from word2vec, but HOW(in Tensorflow)???
    - Chinese Restaurant Process method
    - Gensim doc2vec
    - CNN with multi-genre classification(See Viola's work)
3. Clustering
    - Feed document vectors as features to a classification model
    - Calculate cosine similarity between document vectors, maybe cluster documents using the DBSCAN method?

**Issues:**
1. Both loss and accuracy are increasing during training, why?
    - The loss should decrease as the training process goes on, except for some fluctuation introduced by the mini-batch gradient descent and/or regularization techniques like dropout(that introduces random noise)
    - The accuracy is a measure of how good the predictions of the model are; it should increase during training
    - The validation accuracy decrease indicates that the model is probably overfitting; if both loss and validation accuracy are decreasing, then the model is overfitting
    - If both loss and validation accuracy are increasing, and the loss keeps growing(never decreases), then the model is diverging - there should be something wrong with the hyperparameters, usually you are using a too high learning rate

In [1]:
import numpy as np
import pandas as pd
import codecs
from collections import Counter

In [2]:
imdb_with_storyline = pd.read_csv("../../01_Data/Outputs/imdb_with_storyline.csv")

In [3]:
genres = Counter([item for sublist in list(imdb_with_storyline['genres'].str.split('|')) for item in sublist])
genres = pd.DataFrame(genres.most_common()).set_index([0], drop=True)
del genres.index.name
genres.columns = ['Count']

In [4]:
genres

,Count
Drama,2594
Comedy,1872
Thriller,1411
Action,1153
Romance,1107
Adventure,923
Crime,889
Sci-Fi,616
Fantasy,610
Horror,565


In [5]:
genres.loc[genres['Count']>=500].index.tolist()
# convert other genres to 'Others'

['Drama',
 'Comedy',
 'Thriller',
 'Action',
 'Romance',
 'Adventure',
 'Crime',
 'Sci-Fi',
 'Fantasy',
 'Horror',
 'Family',
 'Mystery']

In [6]:
genres.loc[genres['Count']<500].index.tolist()

['Biography',
 'Animation',
 'Music',
 'War',
 'History',
 'Sport',
 'Musical',
 'Documentary',
 'Western',
 'Film-Noir',
 'Short',
 'News',
 'Reality-TV',
 'Game-Show']

In [9]:
imdb = imdb_with_storyline[['storyline', 'title_year', 'genres']].set_index(imdb_with_storyline['movie_title'])
imdb = pd.concat([imdb[['storyline', 'title_year']], imdb['genres'].apply(lambda x: '|'.join(pd.Series(x))).str.get_dummies()], axis=1)
imdb['Others'] = imdb[genres.loc[genres['Count']<500].index.tolist()].sum(axis=1).apply(lambda x: 1 if x > 0 else 0)
imdb = imdb[['storyline', 'title_year'] + genres.loc[genres['Count']>=500].index.tolist() + ['Others']]

In [10]:
imdb.head()

,storyline,title_year,Drama,Comedy,Thriller,Action,Romance,Adventure,Crime,Sci-Fi,Fantasy,Horror,Family,Mystery,Others
movie_title,,,,,,,,,,,,,,,
Avatar,"When his brother is killed in a robbery, parap...",2009.0,0,0,0,1,0,1,0,1,1,0,0,0,0
Pirates of the Caribbean: At World's End,"After Elizabeth, Will, and Captain Barbossa re...",2007.0,0,0,0,1,0,1,0,0,1,0,0,0,0
Spectre,A cryptic message from the past sends James Bo...,2015.0,0,0,1,1,0,1,0,0,0,0,0,0,0
The Dark Knight Rises,Despite his tarnished reputation after the eve...,2012.0,0,0,1,1,0,0,0,0,0,0,0,0,0
Star Wars: Episode VII - The Force Awakens,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,1


In [11]:
imdb['Others'].value_counts()

0    3736
1    1307
Name: Others, dtype: int64

In [12]:
imdb_train = imdb.loc[imdb['title_year'] < 2015]
imdb_test = imdb.loc[imdb['title_year'] >= 2015]

In [13]:
imdb_train.shape

(4603, 15)

In [14]:
imdb_train = imdb_train.drop("title_year", 1)

In [15]:
imdb_test.shape

(332, 15)

In [16]:
imdb_test = imdb_test.drop("title_year", 1)

In [17]:
imdb.to_csv("../../01_Data/Outputs/storyline_with_genres.csv", index=True)

In [18]:
imdb_train.to_csv("../../01_Data/Outputs/storyline_with_genres_train.csv", index=True)
imdb_test.to_csv("../../01_Data/Outputs/storyline_with_genres_test.csv", index=True)

In [19]:
imdb = pd.read_csv("../../01_Data/Outputs/storyline_with_genres.csv", index_col=0)

In [20]:
imdb.head()

,storyline,title_year,Drama,Comedy,Thriller,Action,Romance,Adventure,Crime,Sci-Fi,Fantasy,Horror,Family,Mystery,Others
movie_title,,,,,,,,,,,,,,,
Avatar,"When his brother is killed in a robbery, parap...",2009.0,0,0,0,1,0,1,0,1,1,0,0,0,0
Pirates of the Caribbean: At World's End,"After Elizabeth, Will, and Captain Barbossa re...",2007.0,0,0,0,1,0,1,0,0,1,0,0,0,0
Spectre,A cryptic message from the past sends James Bo...,2015.0,0,0,1,1,0,1,0,0,0,0,0,0,0
The Dark Knight Rises,Despite his tarnished reputation after the eve...,2012.0,0,0,1,1,0,0,0,0,0,0,0,0,0
Star Wars: Episode VII - The Force Awakens,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,1


In [18]:
imdb['storyline'].to_csv("../../01_Data/Outputs/storyline.txt", sep="\n", index=False)